In [7]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv("https://minio.lab.sspcloud.fr/mligeret1/constructions_non_resid_geocoded.csv",sep=",",on_bad_lines="warn")

In [ ]:
df.head()

,Unnamed: 0,Code de la région du lieu des travaux - Code de la zone,Code de la région du lieu des travaux - Libellé de la zone,Code du département du lieu des travaux - Code de la zone,Code du département du lieu des travaux - Libellé de la zone,Code de la commune du lieu des travaux,"Type de DAU -- demande d'autorisation d'urbanisme (PC ou DP généralement, rarement PA)",Numéro d'enregistrement de la DAU,Etat d'avancement du projet,Date réelle d'autorisation (PC) ou de non-opposition (DP) initiale,...,result_name,result_street,result_postcode,result_city,result_context,result_citycode,result_oldcitycode,result_oldcity,result_district,result_status
0,0,REG_CODE,REG_LIBELLE,DEP_CODE,DEP_LIBELLE,COMM,TYPE_DAU,NUM_DAU,ETAT_DAU,DATE_REELLE_AUTORISATION,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not-found
1,1,82,Rhône-Alpes,01,Ain,01001,PC,00100117V0002,5,2017-05-05,...,L'Abergement-Clémenciat,NaN,1400.0,L'Abergement-Clémenciat,"01, Ain, Auvergne-Rhône-Alpes",01001,NaN,NaN,NaN,ok
2,2,82,Rhône-Alpes,01,Ain,01001,PC,00100119V0005,2,2019-08-29,...,203 Rue des 5 Chemins,Rue des 5 Chemins,1400.0,L'Abergement-Clémenciat,"01, Ain, Auvergne-Rhône-Alpes",01001,NaN,NaN,NaN,ok
3,3,82,Rhône-Alpes,01,Ain,01001,PC,00100120V0003,5,2020-05-25,...,172 Route du Château,Route du Château,1400.0,L'Abergement-Clémenciat,"01, Ain, Auvergne-Rhône-Alpes",01001,NaN,NaN,NaN,ok
4,4,82,Rhône-Alpes,01,Ain,01001,DP,00100119V0011,2,2019-07-22,...,Chemin sous Brody,Chemin sous Brody,1400.0,L'Abergement-Clémenciat,"01, Ain, Auvergne-Rhône-Alpes",01001,NaN,NaN,NaN,ok


In [20]:
for c in df.columns:
    print(c)

Unnamed: 0
Code de la région du lieu des travaux - Code de la zone
Code de la région du lieu des travaux - Libellé de la zone
Code du département du lieu des travaux - Code de la zone
Code du département du lieu des travaux - Libellé de la zone
Code de la commune du lieu des travaux
Type de DAU -- demande d'autorisation d'urbanisme (PC ou DP généralement, rarement PA)
Numéro d'enregistrement de la DAU
Etat d'avancement du projet
Date réelle d'autorisation (PC) ou de non-opposition (DP) initiale
Date réelle d'ouverture de chantier
Date réelle d'achèvement des travaux
Année de dépôt de la DAU
Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation)
Date (mois) de prise en compte (DPC) de l'autorisation initiale
Date (mois) de prise en compte (DPC) de l'ouverture du chantier
Date (mois) (DPC) de dernière mise à jour des données
Code d'activité principale de l'établissement d'un demandeur avéré en tant que personne morale
Catégorie j

In [21]:
df.dtypes

Unnamed: 0                                                        int64
Code de la région du lieu des travaux - Code de la zone          object
Code de la région du lieu des travaux - Libellé de la zone       object
Code du département du lieu des travaux - Code de la zone        object
Code du département du lieu des travaux - Libellé de la zone     object
                                                                 ...   
result_citycode                                                  object
result_oldcitycode                                              float64
result_oldcity                                                   object
result_district                                                  object
result_status                                                    object
Length: 119, dtype: object

In [ ]:

nature_batiments = [x for x in df.columns 
                   if ("issue d'une transformation" in x 
                   or "nouvelle construite" in x)
                   and ("locaux non résidentiels" not in x)]

dates_lieux_travaux = ['Numéro de voie du terrain',
       'Type de voie du terrain', 'Libellé de la voie du terrain',
       'Lieu-dit du terrain', 'Localité du terrain', 'Code postal du terrain','Superficie du terrain',"Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation)"]

colonnes_pertinentes =  dates_lieux_travaux + nature_batiments

In [8]:
df.loc[:,colonnes_pertinentes].sample(150)

,Numéro de voie du terrain,Type de voie du terrain,Libellé de la voie du terrain,Lieu-dit du terrain,Localité du terrain,Code postal du terrain,Superficie du terrain,Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation),Surface de plancher de la destination Habitation nouvelle construite,Surface de plancher de la destination Habitation issue d'une transformation,...,Surface de plancher de la destination Artisanat nouvelle construite,Surface de plancher de la destination Artisanat issue d'une transformation,Surface de plancher de la destination Industrie nouvelle construite,Surface de plancher de la destination Industrie issue d'une transformation,Surface de plancher de la destination Exploitation agricole ou forestière nouvelle construite,Surface de plancher de la destination Exploitation agricole ou forestière issue d'une transformation,Surface de plancher de la destination Entrepôt nouvelle construite,Surface de plancher de la destination Entrepôt issue d'une transformation,Surface de plancher de la destination Services publics ou d'intérêt collectif nouvelle construite,Surface de plancher de la destination Services publics ou d'intérêt collectif issue d'une transformation
56138,NaN,NaN,CHEMEN DU MOULIN BAS,NaN,ARVIEU,12120,540,2021-01,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0
420447,NaN,NaN,NaN,Hambes,Saint-Senier-de-Beuvron,50240.0,73613,2013-04,0.0,0.0,...,0.0,0.0,0.0,0.0,352.0,0.0,0.0,0.0,0.0,0.0
327337,NaN,NaN,LA HAYE D'AMONT,NaN,VAUDREY,39380.0,28760,2023-08,0.0,0.0,...,0.0,0.0,1938.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
365458,NaN,Rue,Marcel Paul Ilot 8f,NaN,NANTES,44000,7993,2019-04,5618.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
775855,14,NaN,AVENUE EUGENE LESPERANT,NaN,MANA,97360,13966,2021-07,0.0,0.0,...,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270322,NaN,NaN,ZAC EUREKA,LES CAUQUILLOUX,Castelnau-le-Lez,34170,97496,2018-07,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
298713,6,NaN,AV DU LION D ARGENT,NaN,MONTGIVRAY,36400,1320,2016-09,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
282220,NaN,NaN,rue du Mont Saint-Michel,NaN,LA BOUSSAC,35120.0,13027,2023-03,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
73368,NaN,NaN,NaN,CONFOUX,LAMANON,13113,1545,2021-04,0.0,61.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,116.0,0.0,0.0


In [15]:
df.loc[:,colonnes_pertinentes].sample(100)


def identification_usage(ligne):
    col_max = ligne.iloc[len(dates_lieux_travaux):].idxmax()
    pattern = r"Surface de plancher de la destination (.+?) (nouvelle construite|issue d'une transformation)$"

    match = re.search(pattern, col_max, re.IGNORECASE)
    print("Testing column:", repr(col_max))
    return match.group(1) if match else None

df_usage = df.loc[:,colonnes_pertinentes].sample(1000)
df_usage["Destination du lieu"]= df_usage.apply(identification_usage, axis = 1)



Testing column: "Surface de plancher de la destination Commerce issue d'une transformation"
Testing column: 'Surface de plancher de la destination Entrepôt nouvelle construite'
Testing column: 'Surface de plancher de la destination Industrie nouvelle construite'
Testing column: "Surface de plancher de la destination Commerce issue d'une transformation"
Testing column: "Surface de plancher de la destination Bureau issue d'une transformation"
Testing column: 'Surface de plancher de la destination Exploitation agricole ou forestière nouvelle construite'
Testing column: 'Surface de plancher de la destination Habitation nouvelle construite'
Testing column: 'Surface de plancher de la destination Habitation nouvelle construite'
Testing column: "Surface de plancher de la destination Bureau issue d'une transformation"
Testing column: 'Surface de plancher de la destination Exploitation agricole ou forestière nouvelle construite'
Testing column: 'Surface de plancher de la destination Hébergement 

TypeError: '>' not supported between instances of 'str' and 'float'